In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
import re
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import gc

c:\Users\imvar\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\imvar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import timm
import transformers

In [3]:
directory = './bully_data'

In [4]:
import pandas as pd

df = pd.read_csv("./Copy of Cyberbully_corrected_emotion_sentiment - cyberbully.csv")
df.head()

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target,Unnamed: 10,Unnamed: 11
0,0.jpg,Shivam @shivamishraa Girls be named naina and ...,Bully,Nonbully,Bully,Negative,Disgust,Yes,Partially-Harmful,Individual,NaN,NaN
1,1.jpg,Aaloo ke paranthe is the best breakfast Omelet...,Nonbully,Nonbully,Nonbully,Neutral,Other,No,Harmless,NaN,NaN,NaN
2,2.jpg,For Boyfriend For Bestfriend DESI ADUKT TROLLS,Bully,Bully,Nonbully,Negative,Ridicule,No,Partially-Harmful,Society,NaN,NaN
3,3.jpg,You find a new YouTuber He's funny All of his ...,Nonbully,Nonbully,Nonbully,Neutral,Sadness,No,Harmless,NaN,NaN,NaN
4,4.jpg,not_shubham14 @mentally_dank Kids at Marine Dr...,Bully,Nonbully,Nonbully,Negative,Sadness,No,Partially-Harmful,Individual,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865 entries, 0 to 5864
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Img_Name        5863 non-null   object
 1   Img_Text        5863 non-null   object
 2   Img_Text_Label  5854 non-null   object
 3   Img_Label       5854 non-null   object
 4   Text_Label      5854 non-null   object
 5   Sentiment       5854 non-null   object
 6   Emotion         5854 non-null   object
 7   Sarcasm         5854 non-null   object
 8   Harmful_Score   5853 non-null   object
 9   Target          3105 non-null   object
 10  Unnamed: 10     4 non-null      object
 11  Unnamed: 11     3 non-null      object
dtypes: object(12)
memory usage: 550.0+ KB


In [6]:
df.dropna(subset=['Img_Text', 'Img_Text_Label'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [7]:
df['Img_Text_Label'].value_counts()

Img_Text_Label
Bully       3222
Nonbully    2632
Name: count, dtype: int64

In [8]:
import os
valid_imgs = os.listdir(directory)
valid_imgs

['0.jpg',
 '1.jpg',
 '10.png',
 '100.png',
 '1000.png',
 '1001.jpg',
 '1002.png',
 '1003.jpg',
 '1004.png',
 '1005.jpg',
 '1006.png',
 '1007.jpg',
 '1008.png',
 '1009.jpg',
 '101.jpg',
 '1010.png',
 '1011.jpg',
 '1012.png',
 '1013.jpg',
 '1014.png',
 '1015.jpg',
 '1016.png',
 '1017.jpg',
 '1018.png',
 '1019.jpg',
 '102.png',
 '1020.png',
 '1021.jpg',
 '1022.png',
 '1023.jpg',
 '1024.png',
 '1025.jpg',
 '1026.png',
 '1027.jpg',
 '1028.png',
 '1029.jpg',
 '103.jpg',
 '1030.png',
 '1031.jpg',
 '1032.png',
 '1033.jpg',
 '1034.png',
 '1035.jpg',
 '1036.png',
 '1037.jpg',
 '1038.png',
 '1039.jpg',
 '104.png',
 '1040.png',
 '1041.jpg',
 '1042.png',
 '1043.jpg',
 '1044.png',
 '1045.jpg',
 '1046.png',
 '1047.jpg',
 '1048.png',
 '1049.jpg',
 '105.jpg',
 '1050.png',
 '1051.jpg',
 '1052.png',
 '1053.jpg',
 '1054.png',
 '1055.jpg',
 '1056.png',
 '1057.jpg',
 '1058.png',
 '1059.jpg',
 '106.png',
 '1060.png',
 '1061.jpg',
 '1062.png',
 '1063.jpg',
 '1064.png',
 '1065.jpg',
 '1066.png',
 '1067.jpg',
 

In [9]:
data = []
labels = {"Nonbully":0, "Bully":1}
for index, row in df.iterrows():
    img_id = row["Img_Name"]
    caption = row["Img_Text"]
    label = row["Img_Text_Label"]
    if img_id in valid_imgs:
        data.append({"ImageID":img_id, "Labels":labels[label], "Caption":caption})

In [10]:
new_df = pd.DataFrame(data)
new_df.head()

,ImageID,Labels,Caption
0,0.jpg,1,Shivam @shivamishraa Girls be named naina and ...
1,1.jpg,0,Aaloo ke paranthe is the best breakfast Omelet...
2,2.jpg,1,For Boyfriend For Bestfriend DESI ADUKT TROLLS
3,3.jpg,0,You find a new YouTuber He's funny All of his ...
4,4.jpg,1,not_shubham14 @mentally_dank Kids at Marine Dr...


In [11]:
train_data, valid_data = train_test_split(new_df, test_size=0.2, random_state=42)
train_data = train_data.reset_index(drop=True)
valid_data = valid_data.reset_index(drop=True)

## Data Preprocessing

In [12]:
from collections import Counter
import nltk

captions = new_df["Caption"].tolist()
caption_tokens = [nltk.word_tokenize(caption.lower()) for caption in captions]
word_counts = Counter(token for caption in caption_tokens for token in caption)

In [13]:
# keep the words that occur more than 5 times
word_counts = {k: v for k, v in word_counts.items() if v>=5}
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
vocab = ['<pad>', '<unk>'] + vocab
word2idx = {word: idx for idx, word in enumerate(vocab)}

In [14]:
len(vocab)

2396

In [15]:
label2idx = {}
i = 1
index = 0
while i <= 19:
    if i == 12:
        i += 1
        continue
    label2idx[str(i)] = index
    index += 1
    i +=1  

In [16]:
MAX_CAPTION_LENGTH = max([len(c) for c in captions])
MAX_CAPTION_LENGTH = 100

### Dataset Creation

In [17]:
class MultiModalDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_id = self.df.loc[index, "ImageID"]
        img_path = f"{directory}/{img_id}"
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        caption = self.df.loc[index, "Caption"]
        caption_tensor = torch.zeros((MAX_CAPTION_LENGTH, ), dtype=torch.long)
        caption_tokens = nltk.word_tokenize(caption.lower())
        for i, token in enumerate(caption_tokens):
            if i >= MAX_CAPTION_LENGTH:
                break
            if token in word2idx:
                caption_tensor[i] = word2idx[token]
            else:
                caption_tensor[i] = word2idx['<unk>'] 
        label = self.df.loc[index, "Labels"]
        label_tensor = torch.tensor([label], dtype=torch.float)

        return image, caption_tensor, label_tensor

In [18]:
IMAGE_SIZE = 224
BATCH_SIZE = 2
lr = 0.001
num_epochs = 5
MAX_CAPTION_LENGTH = 120

In [19]:
transformations = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = MultiModalDataset(train_data, transform=transformations)
val_dataset = MultiModalDataset(valid_data, transform=transformations)

In [20]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, BATCH_SIZE)

In [21]:
for batch in train_loader:
    print(batch)
    break

[tensor([[[[ 0.5059,  0.5059,  0.5216,  ...,  0.5294,  0.5294,  0.5294],
          [-0.9608, -0.9608, -0.9451,  ..., -0.9373, -0.9373, -0.9373],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-0.5765, -0.6235, -0.5843,  ..., -0.5059, -0.7020, -1.0000],
          [-0.5922, -0.6314, -0.6314,  ..., -0.5137, -0.7020, -1.0000],
          [-0.5843, -0.6627, -0.7020,  ..., -0.5137, -0.7020, -1.0000]],

         [[ 0.5373,  0.5373,  0.5373,  ...,  0.5294,  0.5294,  0.5294],
          [-0.9294, -0.9294, -0.9294,  ..., -0.9373, -0.9373, -0.9373],
          [-0.9922, -0.9922, -0.9922,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-0.6000, -0.6392, -0.5686,  ..., -0.5137, -0.7098, -1.0000],
          [-0.6078, -0.6392, -0.6157,  ..., -0.5216, -0.7098, -1.0000],
          [-0.5922, -0.6549, -0.6706,  ..., -0.5216, -0.7098, -1.0000]],

         [[ 0.5294,  0.5294,  0.5294,  ...,  0.5294,  0.5294,  0.5294],
          [-0.9373, -0.9373, 

## Model 

In [27]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
class MultiModalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.image_encoder = timm.create_model('vit_small_r26_s32_224', pretrained=True, num_classes=0, global_pool='')
        self.image_features = nn.Linear(self.image_encoder.num_features, 512)

        self.caption_encoder = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
        self.caption_model = AutoModel.from_pretrained("microsoft/deberta-v3-small")
        self.caption_model.resize_token_embeddings(len(self.caption_encoder))

        self.fc1 = nn.Linear(512 + self.caption_model.config.hidden_size, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 1)
    
    def forward(self, image, caption):
        image_features = self.image_encoder(image)
        image_features = self.image_features(image_features.mean(dim=1))
        image_features = F.relu(image_features)
        image_features = self.dropout(image_features)

        # Convert token IDs to strings
        caption = [self.caption_encoder.convert_ids_to_tokens(caption[i].tolist()) for i in range(len(caption))]
        caption = [' '.join(c) for c in caption]

        # Encoder captions to obtain input_ids and attention_mask tensors
        encoding = self.caption_encoder.batch_encode_plus(caption, padding=True, return_tensors='pt')
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        caption_features = self.caption_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, -1, :]
        features = torch.cat((image_features, caption_features), dim=-1)
        features = self.fc1(features)
        features = self.dropout(features)
        logits = self.fc2(features)
        return logits, features
    
    def predict(self, image_path, caption):
        self.eval()
        device = next(self.parameters()).device
        image = Image.open(image_path).convert('RGB')
        transform = transforms.Compose([
            transforms.resize((224, 224)),
            transform.ToTensor(),
            transform.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
        image = transform(image)
        image = image.unsqueeze(0).to(device)
        caption_inputs = self.caption_encoder(caption, return_tensors='pt')
        caption_inputs = {k: v for k, v in caption_inputs.items()}
        with torch.no_grad():
            logits = self(image, caption_inputs)
            prob = torch.sigmoid(logits).cpu().numpy()
        return prob


### Training

In [28]:
device

device(type='cuda')

In [24]:
!pip install transformers[sentencepiece]

In [29]:
model = MultiModalClassifier().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

c:\Users\imvar\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [30]:
import time
from tqdm import tqdm

NUM_EPOCHS = 5

for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    # set the model to train mode
    model.train()
    train_loss = 0.0
    
    # iterate over the training set
    for batch_idx, (images, captions, labels) in enumerate(tqdm(train_loader)):
        # move data to GPU if available
        images = images.to(device)
        captions = captions.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # forward pass
        logits, features = model(images, captions)

        # calculate loss
        loss = criterion(logits, labels)
        train_loss += loss.item()

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()
        
    # calculate average train loss
    train_loss /= len(train_loader)
    
    # set the model to eval mode
    model.eval()
    val_loss = 0.0

    print("Validation:- ")
    with torch.no_grad():
        val_loss = 0.0
        y_true = []
        y_pred = []

        for batch_idx, (images, captions, labels) in enumerate(tqdm(val_loader)):
            # move data to GPU if available
            images = images.to(device)
            captions = captions.to(device)
            labels = labels.to(device)

            # forward pass
            logits, features = model(images, captions)

            # calculate loss
            loss = criterion(logits, labels)
            val_loss += loss.item()

            # calculate predicted labels
            preds = torch.sigmoid(logits).cpu().numpy()
            preds[preds >= 0.5] = 1
            preds[preds < 0.5] = 0

            # append true and predicted labels
            y_true.append(labels.cpu().numpy())
            y_pred.append(preds)

        # calculate average validation loss
        val_loss /= len(val_loader)

        # calculate macro F1 score
        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)
        f1_macro = f1_score(y_true, y_pred, average='macro')

        # calculate micro F1 score
        f1_micro = f1_score(y_true, y_pred, average='micro')

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train loss: {train_loss:.4f} - Val loss: {val_loss:.4f} - F1 macro: {f1_macro:.4f} - F1 micro: {f1_micro:.4f} - Time: {time.time()-start_time:.2f}s")


100%|██████████| 2319/2319 [06:49<00:00,  5.66it/s]


Validation:- 


100%|██████████| 580/580 [00:18<00:00, 31.00it/s]


Epoch 1/5 - Train loss: 0.7625 - Val loss: 0.7842 - F1 macro: 0.3128 - F1 micro: 0.4552 - Time: 428.31s


100%|██████████| 2319/2319 [06:50<00:00,  5.65it/s]


...

Epoch 4/5 - Train loss: 0.6980 - Val loss: 0.6891 - F1 macro: 0.3527 - F1 micro: 0.5448 - Time: 438.22s


100%|██████████| 2319/2319 [06:49<00:00,  5.66it/s]


Validation:- 


100%|██████████| 580/580 [00:18<00:00, 31.81it/s]

Epoch 5/5 - Train loss: 0.6953 - Val loss: 0.6902 - F1 macro: 0.3527 - F1 micro: 0.5448 - Time: 428.21s
